# Radial Average
## Importing libraries

In [1]:
# Base libraries
import math
import numpy as np
import scipy.integrate as integrate
from tqdm import tqdm
from scipy.special import erf
import pickle
import itertools

from SALib.sample import saltelli
from SALib.analyze import sobol

# Personal libraries
import henon_map as hm

from parameters import *

## Computing

In [2]:
error_2 = {}
DA_2 = {}
count_matrix_2 = {}
average_matrix_2 = {}

alpha_preliminary_values = np.linspace(-1.0, 1.0, samples)
alpha_values = np.arccos(alpha_preliminary_values) / 2
d_preliminar_alpha = alpha_preliminary_values[1] - alpha_preliminary_values[0]

for epsilon in tqdm(epsilons, desc="Radial average"):
    # Extracting the radiuses with theta1 = theta2 = 0.0
    
    engine = hm.radial_scan.generate_instance(
        d_r, 
        alpha_values, 
        np.zeros(alpha_values.shape),
        np.zeros(alpha_values.shape),
        epsilon
    )
    all_radiuses = engine.compute(turn_sampling)
    
    values = []
    for i in tqdm(range(len(turn_sampling))):
        temp_values = np.array([[]])
        for index, j in enumerate(range(0, samples, 128)):
            stopping = (j + 128 if j != samples - 1 else samples)
            radiuses = all_radiuses[j : stopping, i]

            engine = hm.full_track.generate_instance(
                radiuses,
                alpha_values[j : stopping],
                np.zeros(alpha_values.shape)[j : stopping],
                np.zeros(alpha_values.shape)[j : stopping],
                np.ones(alpha_values.shape, dtype=np.int)[j : stopping] * turn_sampling[i],
                epsilon)

            x, y, px, py = engine.compute()
            temp = engine.accumulate_and_return(n_subdivisions)
            tmp_count_matrix = engine.count_matrix
            tmp_avg_matrix = engine.matrices
            if index == 0:
                temp_values = temp
                list_count_matrix = tmp_count_matrix
                list_avg_matrix = tmp_avg_matrix
            else:
                temp_values = np.concatenate((temp_values, temp))
                list_count_matrix = np.concatenate((list_count_matrix, tmp_count_matrix))
                list_avg_matrix = np.concatenate((list_avg_matrix, tmp_avg_matrix))
        
        values.append(temp_values)
        count_matrix_2[(epsilon, samples, turn_sampling[i])] = list_count_matrix
        average_matrix_2[(epsilon, samples, turn_sampling[i])] = list_avg_matrix
    
    steps = [1]
    while True:
        if (values[0].shape[0] - 1) / steps[-1] > 4:
            steps.append(steps[-1] * 2)
        else:
            break

    for jump in steps:
        DA = []
        error = []
        for i in range(len(turn_sampling)):
            DA.append(np.power(integrate.romb(values[i][::jump], d_preliminar_alpha * jump) * 0.5, 1/4))
            temp = np.power(integrate.romb(values[i][::jump * 2], d_preliminar_alpha * jump * 2) * 0.5, 1/4)
            error.append(np.absolute(DA[-1] - temp))
        DA_2[(epsilon, len(values[i][::jump]))] = DA
        error_2[(epsilon, len(values[i][::jump]))] = error

  0%|          | 0/100 [00:00<?, ?it/s]/home/carlidel/anaconda3/envs/work_env/lib/python3.8/site-packages/numpy/lib/function_base.py:393: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/home/carlidel/anaconda3/envs/work_env/lib/python3.8/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)

Radial average: 100%|██████████| 1/1 [04:50<00:00, 290.89s/it]


## Saving data

In [3]:
with open("data/DA_2.pkl", 'wb') as f:
    pickle.dump(DA_2, f)
    
with open("data/error_2.pkl", 'wb') as f:
    pickle.dump(error_2, f)
    
with open("data/count_matrix_2.pkl", 'wb') as f:
    pickle.dump(count_matrix_2, f)
    
with open("data/avg_matrix_2.pkl", 'wb') as f:
    pickle.dump(average_matrix_2, f)